In [1]:
import hail as hl
hl.init()

Running on Apache Spark version 2.4.5
SparkUI available at http://hail-m.us-central1-f.c.cncd-cncd.internal:4042
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.61-3c86d3ba497a
LOGGING: writing to /hail-20210206-1358-0.2.61-3c86d3ba497a.log


In [2]:
recode = {f"{i}":f"chr{i}" for i in (list(range(1, 23)) + ['X', 'Y'])}
mt1 = hl.import_vcf('gs://pgr_wes_30k/GRCh38/chr1_30k_grch38.vcf.bgz', reference_genome='GRCh38', contig_recoding=recode, force_bgz=True)
ds_result = hl.filter_intervals(mt1, [hl.parse_locus_interval('chr1:219173869-219212865', reference_genome='GRCh38')])
print('1')
participant_ids = ['CNCD_V4466_V4466']

ds_result.rows().show()
print(ds_result.count())
ds_result1 = ds_result.filter_cols(hl.literal(participant_ids).contains(ds_result.s))
print(ds_result1.count())
ds_result1.rows().show()

print('2')
ds_result1 = hl.split_multi_hts(ds_result1, permit_shuffle=True)
ds_result1.rows().show()

print('3')
print('Annotate Heterozygous Alleles')
ds_result1 = ds_result1.annotate_rows(hets = hl.agg.filter(ds_result1.GT.is_het(),hl.agg.collect(ds_result1.s)))
ds_result1.count()
print('Annotate N Hets')
ds_result4 = ds_result1.annotate_rows(n_het = hl.agg.count_where(ds_result1.GT.is_het()))
ds_result4.count()
print('Annotate Homozygous Alleles')
ds_result3 = ds_result4.annotate_rows(homs = hl.agg.filter(ds_result4.GT.is_hom_var(),hl.agg.collect(ds_result4.s)))
ds_result3.count()
print('Annotating N Homs')
ds_result5 = ds_result3.annotate_rows(n_hom = hl.agg.count_where(ds_result3.GT.is_hom_var()))

ds_result5.rows().show()

data3 = ds_result5.annotate_rows(
    info=ds_result5.info.annotate(hets=ds_result5.hets,
                             n_het=ds_result5.n_het,
                             homs=ds_result5.homs,
                             n_hom=ds_result5.n_hom)
                                )

hl.export_vcf(data3, "gs://test1haseeb/LYPLAL1_20K.vcf")

1


2021-02-06 13:58:54 Hail: INFO: Coerced sorted dataset
2021-02-06 13:58:54 Hail: INFO: reading 1 of 289 data partitions


+----------------+---------------+------+----------+----------+---------+
| locus          | alleles       | rsid |     qual | filters  | info.AC |
+----------------+---------------+------+----------+----------+---------+
| locus<GRCh38>  | array<str>    | str  |  float64 | set<str> |   int32 |
+----------------+---------------+------+----------+----------+---------+
| chr1:219173875 | ["G","C"]     | NA   | 1.45e+03 | {}       |       1 |
| chr1:219173878 | ["A","C"]     | NA   | 1.24e+04 | {}       |       1 |
| chr1:219173879 | ["G","A"]     | NA   | 1.32e+04 | {}       |       0 |
| chr1:219173879 | ["G","C"]     | NA   | 1.32e+04 | {}       |       1 |
| chr1:219173885 | ["T","A"]     | NA   | 3.47e+03 | {}       |       3 |
| chr1:219173892 | ["T","C"]     | NA   | 7.37e+03 | {}       |       7 |
| chr1:219173904 | ["CGG","C"]   | NA   | 3.34e+03 | {}       |       0 |
| chr1:219173904 | ["CGG","TGG"] | NA   | 3.34e+03 | {}       |       1 |
| chr1:219173905 | ["G","*"]     | NA   | 3.98e+03 | {}       |       0 |
| chr1:219173905 | ["G","A"]     | NA   | 3.98e+03 | {}       |       1 |
+----------------+---------------+------+----------+----------+---------+

+----------+---------------+------------+-----------------------+------------+
| info.aaf | info.callRate | info.hetRA | info.hg19_uid         | info.homAA |
+----------+---------------+------------+-----------------------+------------+
|  float64 |       float64 |      int32 | str                   |      int32 |
+----------+---------------+------------+-----------------------+------------+
| 1.62e-05 |      1.00e+00 |          1 | "1:219347217:G:C"     |          0 |
| 1.62e-05 |      1.00e+00 |          1 | "1:219347220:A:C"     |          0 |
| 0.00e+00 |      1.00e+00 |          0 | "1:219347221:G:A"     |          0 |
| 1.62e-05 |      1.00e+00 |          1 | "1:219347221:G:C"     |          0 |
| 4.86e-05 |      1.00e+00 |          1 | "1:219347227:T:A"     |          1 |
| 1.14e-04 |      1.00e+00 |          7 | "1:219347234:T:C"     |          0 |
| 0.00e+00 |      1.00e+00 |          0 | "1:219347246:CGG:C"   |          0 |
| 1.62e-05 |      1.00e+00 |          1 | "1:219347246:CGG:TGG" |          0 |
| 0.00e+00 |      1.00e+00 |          0 | "1:219347247:G:*"     |          0 |
| 1.62e-05 |      1.00e+00 |          1 | "1:219347247:G:A"     |          0 |
+----------+---------------+------------+-----------------------+------------+

+------------+---------------+---------------+---------------+---------------+
| info.homRR | info.maxDepth | info.maxHetAB | info.maxHomAB | info.medDepth |
+------------+---------------+---------------+---------------+---------------+
|      int32 |         int32 |       float64 |       float64 |         int32 |
+------------+---------------+---------------+---------------+---------------+
|      30831 |           443 |            NA |            NA |            48 |
|      30832 |           443 |            NA |            NA |            48 |
|      30833 |           443 |            NA |            NA |            48 |
|      30832 |           443 |            NA |            NA |            48 |
|      30831 |           443 |            NA |            NA |            48 |
|      30826 |           443 |      4.50e-01 |            NA |            48 |
|      30833 |           443 |            NA |            NA |            48 |
|      30832 |           443 |            NA |            NA |            48 |
|      30833 |           443 |            NA |            NA |            48 |
|      30832 |           443 |            NA |            NA |            48 |
+------------+---------------+---------------+---------------+---------------+

+---------------+---------------+---------------+---------------+
| info.medHetAB | info.medHomAB | info.minDpeth | info.minHetAB |
+---------------+---------------+---------------+---------------+
|       float64 |       float64 |         int32 |       float64 |
+---------------+----

2021-02-06 13:59:29 Hail: INFO: Coerced sorted dataset
2021-02-06 13:59:29 Hail: INFO: reading 1 of 289 data partitions


(236, 30833)


...

2021-02-06 14:02:09 Hail: INFO: Coerced sorted dataset
2021-02-06 14:02:09 Hail: INFO: reading 1 of 289 data partitions
2021-02-06 14:02:19 Hail: INFO: Coerced sorted dataset
2021-02-06 14:02:19 Hail: INFO: reading 1 of 289 data partitions


Annotating N Homs


2021-02-06 14:02:35 Hail: INFO: Coerced sorted dataset
2021-02-06 14:02:35 Hail: INFO: reading 1 of 289 data partitions
2021-02-06 14:02:55 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-02-06 14:03:04 Hail: INFO: Coerced sorted dataset
2021-02-06 14:03:04 Hail: INFO: reading 1 of 289 data partitions


+----------------+-------------+------+----------+----------+---------+
| locus          | alleles     | rsid |     qual | filters  | info.AC |
+----------------+-------------+------+----------+----------+---------+
| locus<GRCh38>  | array<str>  | str  |  float64 | set<str> |   int32 |
+----------------+-------------+------+----------+----------+---------+
| chr1:219173875 | ["G","C"]   | NA   | 1.45e+03 | {}       |       1 |
| chr1:219173878 | ["A","C"]   | NA   | 1.24e+04 | {}       |       1 |
| chr1:219173879 | ["G","A"]   | NA   | 1.32e+04 | {}       |       0 |
| chr1:219173879 | ["G","C"]   | NA   | 1.32e+04 | {}       |       1 |
| chr1:219173885 | ["T","A"]   | NA   | 3.47e+03 | {}       |       3 |
| chr1:219173892 | ["T","C"]   | NA   | 7.37e+03 | {}       |       7 |
| chr1:219173904 | ["C","T"]   | NA   | 3.34e+03 | {}       |       1 |
| chr1:219173904 | ["CGG","C"] | NA   | 3.34e+03 | {}       |       0 |
| chr1:219173905 | ["G","A"]   | NA   | 3.98e+03 | {}       |       1 |
| chr1:219173905 | ["G","C"]   | NA   | 3.98e+03 | {}       |       0 |
+----------------+-------------+------+----------+----------+---------+

+----------+---------------+------------+-----------------------+------------+
| info.aaf | info.callRate | info.hetRA | info.hg19_uid         | info.homAA |
+----------+---------------+------------+-----------------------+------------+
|  float64 |       float64 |      int32 | str                   |      int32 |
+----------+---------------+------------+-----------------------+------------+
| 1.62e-05 |      1.00e+00 |          1 | "1:219347217:G:C"     |          0 |
| 1.62e-05 |      1.00e+00 |          1 | "1:219347220:A:C"     |          0 |
| 0.00e+00 |      1.00e+00 |          0 | "1:219347221:G:A"     |          0 |
| 1.62e-05 |      1.00e+00 |          1 | "1:219347221:G:C"     |          0 |
| 4.86e-05 |      1.00e+00 |          1 | "1:219347227:T:A"     |          1 |
| 1.14e-04 |      1.00e+00 |          7 | "1:219347234:T:C"     |          0 |
| 1.62e-05 |      1.00e+00 |          1 | "1:219347246:CGG:TGG" |          0 |
| 0.00e+00 |      1.00e+00 |          0 | "1:219347246:CGG:C"   |          0 |
| 1.62e-05 |      1.00e+00 |          1 | "1:219347247:G:A"     |          0 |
| 0.00e+00 |      1.00e+00 |          0 | "1:219347247:G:C"     |          0 |
+----------+---------------+------------+-----------------------+------------+

+------------+---------------+---------------+---------------+---------------+
| info.homRR | info.maxDepth | info.maxHetAB | info.maxHomAB | info.medDepth |
+------------+---------------+---------------+---------------+---------------+
|      int32 |         int32 |       float64 |       float64 |         int32 |
+------------+---------------+---------------+---------------+---------------+
|      30831 |           443 |            NA |            NA |            48 |
|      30832 |           443 |            NA |            NA |            48 |
|      30833 |           443 |            NA |            NA |            48 |
|      30832 |           443 |            NA |            NA |            48 |
|      30831 |           443 |            NA |            NA |            48 |
|      30826 |           443 |      4.50e-01 |            NA |            48 |
|      30832 |           443 |            NA |            NA |            48 |
|      30833 |           443 |            NA |            NA |            48 |
|      30832 |           443 |            NA |            NA |            48 |
|      30833 |           443 |            NA |            NA |            48 |
+------------+---------------+---------------+---------------+---------------+

+---------------+---------------+---------------+---------------+
| info.medHetAB | info.medHomAB | info.minDpeth | info.minHetAB |
+---------------+---------------+---------------+---------------+
|       float64 |       float64 |         int32 |       float64 |
+---------------+---------------+---------------+----

2021-02-06 14:03:45 Hail: WARN: export_vcf: ignored the following fields:
    'a_index' (row)
    'was_split' (row)
    'hets' (row)
    'n_het' (row)
    'homs' (row)
    'n_hom' (row)
2021-02-06 14:04:01 Hail: INFO: Coerced sorted dataset
2021-02-06 14:04:01 Hail: INFO: reading 1 of 289 data partitions
2021-02-06 14:04:37 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-02-06 14:04:53 Hail: INFO: Coerced sorted dataset
2021-02-06 14:04:53 Hail: INFO: reading 1 of 289 data partitions
2021-02-06 14:06:10 Hail: INFO: merging 1 files totalling 273.8K...
2021-02-06 14:06:11 Hail: INFO: while writing:
    gs://test1haseeb/LYPLAL1_20K.vcf
  merge time: 353.538ms
